**Это код, который собирает сообщения за конкретное время (устанавливается самостоятельно) от всех пользователей**

In [68]:
!pip install telethon
import asyncio
from telethon import TelegramClient
from telethon.errors import FloodWaitError
from telethon.tl.functions.messages import GetHistoryRequest
from telethon.tl.types import PeerUser, PeerChannel
import csv
import datetime

# Ваши данные API
api_id =
api_hash = ''
phone = ''
session_name = 'all_chats'

# Имена или ID чатов для парсинга
chat_names = [
    'teletoloka',
    'RussianFreelance',
    'freelancer_boost',
    'workzavr',
    'freelancechatikk',
    'digital_partners_chat',
    'marketing_chats'
]

# Списки ключевых слов
keywords_list_1 = [
    "ищу", "ищем", "требуется", "требуются", "нужен", "нужны", "в поиске",
    "в поисках", "lookfor", "сотрудничество", "сотрудничества", "подрядчик",
    "подрядчика", "подрядчиков", "специалист", "специалиста", "специалистов"
]
keywords_list_2 = [
    "контекст", "контекстная реклама", "таргет", "таргетированная реклама", "seo",
    "digital-маркетинг", "digital маркетинг", "диджитал маркетинг", "диджитал-маркетинг",
    "digital-маркетолог", "digital маркетолог", "диджитал маркетолог", "диджитал-маркетолог",
    "интернет маркетолог", "интернет маркетинг", "интернет-маркетолог", "интернет-маркетинг",
    "директолог", "ppc", "Яндекс Директ", "Яндекс.Директ", "Гугл Адс", "Гугл Эдс",
    "Google Ads", "Яндекс Метрика", "Яндекс.Метрика", "Google Analytics","исследование",
    "Гугл Карты","Google Карты","Google Maps","Яндекс Карты","Яндекс.Карты","Яндекс Бизнес","Яндекс.Бизнес",
]

# Список минус-слов
exclude_keywords = [
    "освободился", "освободились", "свободен","свободны","свободные","доступен","доступны","предлагаем","сделаем"
]

client = TelegramClient(session_name, api_id, api_hash, timeout=10)

# Устанавливаем начальную дату (5 дней) с временной зоной UTC
start_date = datetime.datetime.now(datetime.timezone.utc) - datetime.timedelta(days=5)

async def fetch_messages_with_keywords(chat_name):
    await client.start()
    print(f"Подключено к Telegram для чата {chat_name}")

    chat = await client.get_entity(chat_name)
    all_messages = []
    offset_id = 0
    limit = 300
    max_messages = 500
    user_data_cache = {}

    while True:
        try:
            print(f"Получение сообщений с offset_id: {offset_id}")
            history = await client(GetHistoryRequest(
                peer=chat,
                offset_id=offset_id,
                offset_date=None,
                add_offset=0,
                limit=limit,
                max_id=0,
                min_id=0,
                hash=0
            ))

            if not history.messages:
                print("Больше сообщений не найдено.")
                break

            for message in history.messages:
                # Проверяем дату сообщения, чтобы закончить, когда достигнем сообщений старше 1 дня
                if message.date < start_date:
                    print("Достигнуты сообщения старше одного дня.")
                    return all_messages

                if message.message:
                    contains_keyword_1 = any(keyword.lower() in message.message.lower() for keyword in keywords_list_1)
                    contains_keyword_2 = any(keyword.lower() in message.message.lower() for keyword in keywords_list_2)
                    contains_exclude_keyword = any(exclude.lower() in message.message.lower() for exclude in exclude_keywords)

                    if contains_keyword_1 and contains_keyword_2 and not contains_exclude_keyword:
                        if isinstance(message.from_id, PeerUser):
                            user_id = message.from_id.user_id
                            if user_id not in user_data_cache:
                                sender = await client.get_entity(user_id)
                                username = sender.username or "N/A"
                                name = f"{sender.first_name or ''} {sender.last_name or ''}".strip()
                                user_data_cache[user_id] = (username, name)
                            username, name = user_data_cache[user_id]
                        elif isinstance(message.from_id, PeerChannel):
                            user_id = "Channel"
                            username = "N/A"
                            name = "Channel Message"

                        #print(f"Сообщение найдено: {message.message[:50]}...")  # Вывод первых 50 символов сообщения
                        all_messages.append({
                            'User ID': user_id,
                            'Date': message.date.strftime('%Y-%m-%d %H:%M:%S'),
                            'Username': username,
                            'Name': name,
                            'Message': message.message,
                        })

                        if len(all_messages) >= max_messages:
                            print(f"Достигнут лимит сообщений: {max_messages}")
                            return all_messages

            offset_id = history.messages[-1].id
            await asyncio.sleep(3)  # Пауза на 3 секунды между запросами

        except FloodWaitError as e:
            wait_time = e.seconds + 30  # Добавляем дополнительные 30 секунд к указанному Telegram времени
            print(f"FloodWaitError: ожидание {wait_time} секунд.")
            await asyncio.sleep(wait_time)

        except ConnectionError:
            print("Соединение потеряно. Повторное подключение...")
            await client.disconnect()
            await asyncio.sleep(5)
            await client.connect()

    await client.disconnect()
    return all_messages

# Сохранение сообщений в CSV-файл
def save_messages_to_csv(messages, filename='/content/filtered_messages_TG.csv'):
    try:
        with open(filename, mode='w', encoding='utf-8', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=['User ID','Date', 'Username', 'Name', 'Message'])
            writer.writeheader()
            writer.writerows(messages)
        print(f"Сообщения успешно сохранены в {filename}")
    except Exception as e:
        print(f"Ошибка при сохранении CSV: {e}")


# Основная функция
async def main():
    all_chat_messages = []

    for chat_name in chat_names:
        messages = await fetch_messages_with_keywords(chat_name)
        all_chat_messages.extend(messages)

    # Сохраняем все собранные сообщения в CSV
    save_messages_to_csv(all_chat_messages)

# Запуск асинхронного кода
await main()






Подключено к Telegram для чата teletoloka
Получение сообщений с offset_id: 0
Получение сообщений с offset_id: 1975288
Достигнуты сообщения старше одного дня.
Подключено к Telegram для чата RussianFreelance
Получение сообщений с offset_id: 0
Получение сообщений с offset_id: 1513652
Получение сообщений с offset_id: 1513549
Получение сообщений с offset_id: 1513444
Получение сообщений с offset_id: 1513339
Получение сообщений с offset_id: 1513232
Получение сообщений с offset_id: 1513124
Получение сообщений с offset_id: 1513014
Получение сообщений с offset_id: 1512904
Получение сообщений с offset_id: 1512794
Получение сообщений с offset_id: 1512683
Получение сообщений с offset_id: 1512572
Получение сообщений с offset_id: 1512461
Получение сообщений с offset_id: 1512354
Получение сообщений с offset_id: 1512251
Получение сообщений с offset_id: 1512147
Получение сообщений с offset_id: 1512042
Получение сообщений с offset_id: 1511938
Получение сообщений с offset_id: 1511835
Получение сообщений с

**Запускать после каждой итерации парсинга, чтобы закрыть сессию и избежать блокировок**

In [80]:
# Завершение клиента и пауза
await client.disconnect()
await asyncio.sleep(1)  # Небольшая пауза для завершения фоновых задач

**Загружаем наши данные в датасет**

In [74]:
import pandas as pd
df = pd.read_csv('filtered_messages_TG.csv')
df.head(60)
#df.shape

,User ID,Date,Username,Name,Message
0,964038097,2024-11-08 08:18:49,v_detsina,Vlad Detsina,#ищу #таргетолог\n\nПривет) \n\nНужен таргетол...
1,1084784310,2024-11-08 06:20:36,nastyap0302,Настя,Хотите максимально эффективно продвигать свой ...
2,299721796,2024-11-07 12:06:02,mistavlado_target,Владислав | Реклама Instagram VK🎯,#помогу #таргетолог #таргет #рекламаинстаграм ...
3,6236116256,2024-11-07 11:55:26,Lina_varismm,Лина VARI.smm,#ищу #ассистент #таргетолог\nПривет!\n\nЯ ищу ...
4,1051153167,2024-11-07 11:18:31,E_Zhivova,Екатерина Живова,#Ищу #Маркетолог\n\n🔴Мы ищем опытного маркетол...
5,6441283563,2024-11-07 05:54:46,Mari_Efromeeva,Марина Е.,"#помогу \n\nМеня зовут Марина, я технический с..."
6,6975245401,2024-11-06 18:48:13,AlinaHelper,Алина ∆ Помощник Hr-Специалиста,Всем привет!! 😎\n\n Срочно нужен дизайнер!!!!\...
7,700124412,2024-11-06 11:03:42,katerinamika90,Катерина,#помогу #таргетолог #smm\n⠀\n💥💥💥Smm и таргет з...
8,5088236603,2024-11-06 06:37:12,Marina_Efromeeva,Марина Е.,"#помогу \n\nМеня зовут Марина, я технический с..."
9,783953433,2024-11-05 08:49:58,bolbelmafia,Петр Сысоев,#ищу #пинтерест #продвижениеютуб #продюсер #ют...


**Оставляем строки без пропущенных данных**

In [79]:
import numpy as np
#df['Username'] = df['Username'].replace("Пусто", np.nan)

# Удаляем строки с NaN в столбце Username
df_filtered = df.dropna(subset=['Username'])
df_filtered.head(-30)
#df_filtered.shape

,User ID,Date,Username,Name,Message
0,964038097,2024-11-08 08:18:49,v_detsina,Vlad Detsina,#ищу #таргетолог\n\nПривет) \n\nНужен таргетол...
1,1084784310,2024-11-08 06:20:36,nastyap0302,Настя,Хотите максимально эффективно продвигать свой ...
2,299721796,2024-11-07 12:06:02,mistavlado_target,Владислав | Реклама Instagram VK🎯,#помогу #таргетолог #таргет #рекламаинстаграм ...
3,6236116256,2024-11-07 11:55:26,Lina_varismm,Лина VARI.smm,#ищу #ассистент #таргетолог\nПривет!\n\nЯ ищу ...
4,1051153167,2024-11-07 11:18:31,E_Zhivova,Екатерина Живова,#Ищу #Маркетолог\n\n🔴Мы ищем опытного маркетол...
...,...,...,...,...,...
451,6067128361,2024-11-07 11:49:46,max_azaradv,Максим РЕКЛАМА,#помогу #настройка #реклама #директолог \n \nЗ...
452,510428623,2024-11-07 09:52:18,MA_PROdavai,Команда «ПРОДАВАЙ»,"Мы, агентство по продвижению и привлечению тра..."
453,1141609077,2024-11-07 09:29:46,brikina,Алина I Таргетолог,#помогу #таргетолог #реклама \n\nИщу людей для...
454,6051661213,2024-11-06 13:29:40,maxim_egor,Maxim Egorov,#помогу #настройка #реклама #директолог \n \nЗ...


**Скачиваем файл**

In [78]:
# Сохраняем DataFrame как CSV
df_filtered.to_csv('df_filtered.csv', index=False)

# Скачиваем файл
files.download('df_filtered.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>